In [1]:
from sys import displayhook
from binance.client import Client
from binance.enums import *
import pandas as pd
import time
import datetime
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import seaborn as sns

api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)

db_data = 'mysql+mysqldb://' + 'admin' + ':' + '0323A8E3DB' + '@' + 'coin-database-1-instance-1.c9hqcydvu19f.us-east-1.rds.amazonaws.com' + ':3306/' + 'crypto_db' + '?charset=utf8mb4'
engine = create_engine(db_data)

cryptoString = 'SELECT * FROM crypto'
crypto_table = pd.read_sql_query(cryptoString, engine)
tickerdict = pd.Series(crypto_table.ticker.values,index=crypto_table.id).to_dict()

In [2]:
# start_string_close = "2022-03-01 12:00:00" 
# start_string = "2022-03-01 14:00:00"
# end_string = "2022-04-15 21:00:00"
# start_date = datetime.datetime.strptime(start_string, '%Y-%m-%d %H:%M:%S')
# end_date = datetime.datetime.strptime(end_string, '%Y-%m-%d %H:%M:%S')

# new_close_string = f"SELECT crypto_id, date, close FROM crypto_price WHERE date >= '{start_string_close}' AND date <= '{end_string}' ORDER BY date"
# closes_all = pd.read_sql_query(new_close_string, engine)

# closes_all.to_pickle("./pickles/closes_all_110322_030422.pkl")

# new_trends_string = f"SELECT * FROM crypto_trend WHERE date_request > '{start_string}' AND date_request < '{end_string}' ORDER BY date_request, crypto_id, date"
# new_trends_all = pd.read_sql_query(new_trends_string, engine)

# new_trends_all.to_pickle("./pickles/new_trends_all_010322_150422.pkl") 

In [3]:
closes_all = pd.read_pickle("./pickles/closes_all_010322_150422.pkl")
new_trends_all = pd.read_pickle("./pickles/new_trends_all_010322_150422.pkl") 

In [4]:
with open('./pickles/output_results_2204.pickle', 'rb') as handle:
    output_results = pkl.load(handle)

In [5]:
output_results_todf = []

for x in range(len(output_results)):
    for y in range(len(output_results[x][1])):
        output_results_todf.append( [output_results[x][0][0], output_results[x][0][1] ] + list(output_results[x][1][y]) )

In [6]:
df = pd.DataFrame(output_results_todf)
df.rename(columns={0: 'sbuy', 1: 'ssell', 2: 'id', 3: 'tbuy', 4: 'tsell', 5: 'pbuy', 6: 'psell', 7: 'ret',}, inplace=True)

In [7]:
df_group = df.groupby(['id', 'tbuy', 'tsell']).max().sort_values(by=['id', 'tbuy'], ascending=[True, False])

In [8]:
arr = np.array(df_group.reset_index())
arr[0]

array([1, Timestamp('2022-03-31 02:00:00'),
       Timestamp('2022-03-31 03:00:00'), 80, 80, 1.904, 1.896,
       0.9957983193277311], dtype=object)

In [10]:
def numpy_ewma_vectorized(data, window):

    alpha = 2 /(window + 1.0)
    alpha_rev = 1-alpha

    scale = 1/alpha_rev
    n = data.shape[0]

    r = np.arange(n)
    scale_arr = scale**r
    offset = data[0]*alpha_rev**(r+1)
    pw0 = alpha*alpha_rev**(n-1)

    mult = data*pw0*scale_arr
    cumsums = mult.cumsum()
    out = offset + cumsums*scale_arr[::-1]
    return out

In [26]:
windowSize = 20



C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_6672\1354611316.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close['ewma'] = close['close'].ewm(span=windowSize, min_periods=windowSize).mean().values


In [32]:
pd.options.mode.chained_assignment = None
id = 1
for id in list(set(arr[:,0]))[0:5]:
#for id in [1]:
    for trade in arr[(arr[:,0] == id)]:
        buy_trends = new_trends_all.loc[(new_trends_all['date_request'] > str(trade[1])) & 
                                        (new_trends_all['date_request'] < str(trade[1] + datetime.timedelta(hours=1))) & 
                                        (new_trends_all['crypto_id'] == id)]
        sell_trends = new_trends_all.loc[(new_trends_all['date_request'] > str(trade[2])) & 
                                        (new_trends_all['date_request'] < str(trade[2] + datetime.timedelta(hours=1))) & 
                                        (new_trends_all['crypto_id'] == id)]

        close = closes_all.loc[(closes_all['date'] >= str(trade[1] - datetime.timedelta(hours=10))) & 
                                        (closes_all['date'] <= str(trade[1] + datetime.timedelta(hours=10))) & 
                                        (closes_all['crypto_id'] == id)]
        close['ewma'] = close['close'].ewm(span=windowSize, min_periods=windowSize).mean()

        if buy_trends.empty:
            print(f"buy empty for {trade}")
            continue
        if sell_trends.empty:
            print(f"sell empty for {trade}")
            continue
        if close.empty:
            print(f"close empty for {trade}")
            continue

        # Figure setup
        fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize=(16,16), constrained_layout=True, gridspec_kw={'height_ratios': [1, 1, 1.6]})

        ax0.axhline(y = buy_trends[buy_trends['hype'] !=0]['hype'].quantile(0.75), color = 'lavender', label = '75% NZ')
        ax0.axhline(y = buy_trends['hype'].quantile(0.75), color = 'lightsteelblue', label = '75%')

        # Plotting buy
        buy_trends.plot( x = 'date', y='hype' , label='buy trend'  , color='g', ax = ax0)
        ax0.axvline(x = buy_trends[buy_trends['hype'] == 100]['date'].item(), color = 'b', label = '100')
        
        ax0.legend()
        ax0.set_title('buy ind')

        # Plotting Sell
        sell_trends.plot(x = 'date', y='hype' , legend='buy trend'  , color='r'       , ax = ax1)
        ax1.set_title('sell ind')

        # Plotting close 
        close.plot(      x = 'date', y=['close', 'ewma'], legend='close price', color=['fuchsia', 'coral']    , ax = ax2)
        ax2.axvline(x = buy_trends[buy_trends['hype'] == 100]['date'].item(), color = 'b', label = '100')
        ax2.axvline(x = buy_trends.iloc[0]['date_request'], color = 'g', label = 'buy')
        ax2.axvline(x = sell_trends.iloc[0]['date_request'], color = 'r', label = 'sell')
        ax2.legend()
        ax2.set_title('close')

        fig.suptitle(f'Buy, Sell indicator & close for coin {tickerdict[id]}. P/L = {trade[7]}', fontsize=16)

        if trade[7] > 1:
            pl = "P"
            fig.savefig(f"./plots/p/{tickerdict[id]}-{buy_trends.iloc[0]['date_request'].strftime('D%Y_%m_%dT%H_%M_%S')}-{sell_trends.iloc[0]['date_request'].strftime('D%Y_%m_%dT%H_%M_%S')}-{pl}.png")

        else:
            pl = "L"
            fig.savefig(f"./plots/l/{tickerdict[id]}-{buy_trends.iloc[0]['date_request'].strftime('D%Y_%m_%dT%H_%M_%S')}-{sell_trends.iloc[0]['date_request'].strftime('D%Y_%m_%dT%H_%M_%S')}-{pl}.png")

        plt.close()

In [ ]:
buy_trends.empty

False

In [ ]:
new_trends_all

,crypto_id,date_request,date,hype
0,101,2022-03-11 14:02:03,2022-03-04 15:00:00,82.0
1,101,2022-03-11 14:02:03,2022-03-04 16:00:00,88.0
2,101,2022-03-11 14:02:03,2022-03-04 17:00:00,90.0
3,101,2022-03-11 14:02:03,2022-03-04 18:00:00,87.0
4,101,2022-03-11 14:02:03,2022-03-04 19:00:00,89.0
...,...,...,...,...
28484623,304,2022-04-03 20:11:44,2022-04-03 16:00:00,80.0
28484624,304,2022-04-03 20:11:44,2022-04-03 17:00:00,58.0
28484625,304,2022-04-03 20:11:44,2022-04-03 18:00:00,39.0
28484626,304,2022-04-03 20:11:44,2022-04-03 19:00:00,47.0


In [ ]:
trade[1]

Timestamp('2022-03-11 04:00:00')

In [ ]:
df.iloc[2]['No_Of_Units']

In [ ]:
buy_trends.at[100, 'hype']

KeyError: 100

In [ ]:
df['tsell'].max()

Timestamp('2022-04-14 20:00:00')

In [ ]:
ax0.axvline(x = buy_trends[buy_trends['hype'] == 100][['date']], color = 'b', label = 'buy')

In [ ]:
trade[5]

array([1, Timestamp('2022-03-31 02:00:00'),
       Timestamp('2022-03-31 03:00:00'), 80, 80, 1.904, 1.896,
       0.9957983193277311], dtype=object)

In [ ]:
sell_trends.iloc[0]['date_request']

Timestamp('2022-03-31 03:02:04')

In [ ]:
np.interp(1.904, close['date'], close['close'])

1.852